## City of Chicago Bus Tracker Kafka Producer

Login to the CTA portal and Create a developer API Key: 

https://www.transitchicago.com/developers/bustracker/

In [7]:
#!pip install kafka-python
#!pip install ctabustracker

In [1]:
import random 
from time import sleep
from datetime import datetime 
from json import dumps
from kafka import KafkaProducer, KafkaAdminClient

In [15]:
KAFKA_TOPIC = "bustrack"
KAFKA_BROKER = ['127.0.0.1:9092', '192.168.99.100:9092']
GROUP = 'my-group'
API_KEY='YOUR_API_KEY'

## CTA Bus Tracker API

In [16]:
from ctabustracker import CTABusTracker

cbt = CTABusTracker('tjRNZR94w97ip25AdR68UYUvF')

In [17]:
#get all CTA routes from the API
cbt.get_routes()

{'1': {'id': '1', 'name': 'Bronzeville/Union Station'},
 '2': {'id': '2', 'name': 'Hyde Park Express'},
 '3': {'id': '3', 'name': 'King Drive'},
 '4': {'id': '4', 'name': 'Cottage Grove'},
 'X4': {'id': 'X4', 'name': 'Cottage Grove Express'},
 '5': {'id': '5', 'name': 'South Shore Night Bus'},
 '6': {'id': '6', 'name': 'Jackson Park Express'},
 '7': {'id': '7', 'name': 'Harrison'},
 '8': {'id': '8', 'name': 'Halsted'},
 '8A': {'id': '8A', 'name': 'South Halsted'},
 '9': {'id': '9', 'name': 'Ashland'},
 'X9': {'id': 'X9', 'name': 'Ashland Express'},
 '11': {'id': '11', 'name': 'Lincoln'},
 '12': {'id': '12', 'name': 'Roosevelt'},
 'J14': {'id': 'J14', 'name': 'Jeffery Jump'},
 '15': {'id': '15', 'name': 'Jeffery Local'},
 '18': {'id': '18', 'name': '16th/18th'},
 '19': {'id': '19', 'name': 'United Center Express'},
 '20': {'id': '20', 'name': 'Madison'},
 '21': {'id': '21', 'name': 'Cermak'},
 '22': {'id': '22', 'name': 'Clark'},
 '24': {'id': '24', 'name': 'Wentworth'},
 '26': {'id': '

In [18]:
#select route id from user input
route_id = input("Input the route you would like to track: ")

#get all vehicles going to route
response = cbt.get_route_vehicles(route_id=route_id)
response

{'8402': {'id': '8402',
  'last_update': datetime.datetime(2024, 2, 8, 16, 43),
  'latitude': '41.78759822845459',
  'longitude': '-87.59150695800781',
  'heading': 359,
  'pattern_id': '14103',
  'route_id': '172',
  'destination': 'Hyde Park/East End',
  'distance_into_route': 2184.0,
  'delayed': False},
 '8491': {'id': '8491',
  'last_update': datetime.datetime(2024, 2, 8, 16, 42),
  'latitude': '41.80253982543945',
  'longitude': '-87.58412170410156',
  'heading': 273,
  'pattern_id': '14110',
  'route_id': '172',
  'destination': '60th/University',
  'distance_into_route': 48.0,
  'delayed': False},
 '8444': {'id': '8444',
  'last_update': datetime.datetime(2024, 2, 8, 16, 43),
  'latitude': '41.80240013485863',
  'longitude': '-87.59295872279576',
  'heading': 269,
  'pattern_id': '14110',
  'route_id': '172',
  'destination': '60th/University',
  'distance_into_route': 2474.0,
  'delayed': False}}

In [19]:
#select a vehicle id from the route
vehicle_id = input("Input the vehicle you would like to track: ")

## Kafka Producer


In [20]:
import datetime
def dateconverter(o):
    if isinstance(o, datetime.datetime):
        return o.isoformat()

In [22]:
producer = KafkaProducer(bootstrap_servers= KAFKA_BROKER,
                         value_serializer=lambda x: 
                         dumps(x, default=dateconverter).encode('utf-8'))

In [23]:
#call the webservice once every 10-20 seconds
for e in range(200):
    response = cbt.get_vehicles(vehicle_ids=[vehicle_id])
    print("Bus Status: ", response)
    producer.send(KAFKA_TOPIC, value=response)
    sleep(random.randint(10,20))
producer.close()

Bus Status:  [{'id': '8491', 'last_update': datetime.datetime(2024, 2, 8, 16, 43, 23), 'latitude': '41.80253982543945', 'longitude': '-87.58412170410156', 'heading': 273, 'pattern_id': '14110', 'route_id': '172', 'destination': '60th/University', 'distance_into_route': 48.0, 'delayed': False}]
Bus Status:  [{'id': '8491', 'last_update': datetime.datetime(2024, 2, 8, 16, 43, 53), 'latitude': '41.80253982543945', 'longitude': '-87.58412170410156', 'heading': 273, 'pattern_id': '14110', 'route_id': '172', 'destination': '60th/University', 'distance_into_route': 48.0, 'delayed': False}]
Bus Status:  [{'id': '8491', 'last_update': datetime.datetime(2024, 2, 8, 16, 43, 53), 'latitude': '41.80253982543945', 'longitude': '-87.58412170410156', 'heading': 273, 'pattern_id': '14110', 'route_id': '172', 'destination': '60th/University', 'distance_into_route': 48.0, 'delayed': False}]
Bus Status:  [{'id': '8491', 'last_update': datetime.datetime(2024, 2, 8, 16, 44, 23), 'latitude': '41.802539825439

KeyboardInterrupt: 